In [12]:
import pandas as pd
import numpy as np
import math
import sklearn
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow
import keras

In [13]:

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [38]:
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [5]:
# settings

# define combinations that are to be tested
combinations = [(["r","g","b"],"RGB"),(["r","g","b","dsm"],"RGB+DSM"),
               (["r","g","b","re"],"RGB+RE"),(["r","g","b","nir"],"RGB+NIR"),
               (["r","g","b","dsm","nir","re"],"RGB+DSM+NIR+RE")]
input_data_path = "sampled_test_exhaustive.csv"

## Import Data

In [6]:
df = pd.read_csv(input_data_path,low_memory=False)

In [7]:
df

,x,y,test,train,merged_5_1,merged_5_2,merged_5_3,merged_5_4,merged_5_5,merged_5_6,merged_5_7,merged_5_8,merged_5_9
0,319903.8164,6843727.743,bush,NaN,-35.74009,0.0,0.0,0.0,0.0,88.0,72.0,77.0,255.0
1,319903.8664,6843727.743,bush,NaN,-35.74701,0.0,0.0,0.0,0.0,32.0,24.0,29.0,255.0
2,319903.9164,6843727.743,bush,NaN,-35.75975,0.0,0.0,0.0,0.0,8.0,7.0,8.0,255.0
3,319903.9664,6843727.743,bush,NaN,-35.76017,0.0,0.0,0.0,0.0,39.0,18.0,31.0,255.0
4,319904.0164,6843727.743,bush,NaN,-35.77869,0.0,0.0,0.0,0.0,13.0,15.0,16.0,255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280218,319897.2171,6843615.508,grass,NaN,-36.70755,5758.0,65535.0,6984.0,65535.0,50.0,55.0,60.0,255.0
3280219,319897.2671,6843615.508,grass,NaN,-36.71937,5951.0,65535.0,6861.0,65535.0,51.0,53.0,59.0,255.0
3280220,319897.3171,6843615.508,grass,NaN,-36.75003,5637.0,65535.0,7047.0,65535.0,77.0,72.0,62.0,255.0
3280221,319897.3671,6843615.508,grass,NaN,-36.74512,5473.0,65535.0,7917.0,65535.0,16.0,20.0,12.0,255.0


In [8]:
df.columns = ["x","y","test","train","dsm","re","re2","nir","nir2","r","g","b","alpha"]

In [9]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,x,y,dsm,re,re2,nir,nir2,r,g,b,alpha
count,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000,3280223.00000
mean,319898.18840,6843672.69140,-36.41859,16029.59990,64460.19917,17902.60827,64655.05265,126.13685,127.22482,108.11254,255.00000
std,23.17302,29.99977,1.30195,9530.10157,8323.57475,10157.78073,7542.74881,66.84854,63.12405,59.65089,0.00000
min,319847.27180,6843615.50800,-38.38610,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000
25%,319879.61870,6843647.25400,-37.38474,9006.00000,65535.00000,10209.00000,65535.00000,68.00000,71.00000,56.00000,255.00000
50%,319898.26700,6843672.75000,-36.50815,14972.00000,65535.00000,16978.00000,65535.00000,129.00000,129.00000,104.00000,255.00000
75%,319916.66520,6843698.29700,-36.06096,21377.00000,65535.00000,23918.00000,65535.00000,183.00000,181.00000,157.00000,255.00000
max,319950.36200,6843727.74300,-26.23309,65535.00000,65535.00000,65535.00000,65535.00000,255.00000,255.00000,255.00000,255.00000


## Prepare Data

In [119]:
# clean up target columns
df_train = df["train"]
tmp = []
for i in df_train:
    a = str(i)
    if a == "nan":
        tmp.append("")
    else:
        tmp.append(i)
df["train"] = tmp

df_test = df["test"]
tmp = []
for i in df_test:
    a = str(i)
    if a == "nan":
        tmp.append("")
    else:
        tmp.append(i)
df["test"] = tmp

# extract train and test datasets
df_train = df[df["train"] != ""]
df_test = df[df["test"] != ""]

## Define Functions

In [120]:
def workflow(columns,title,data):

    
    # extract train data
    x_train = data[data["train"] != ""][columns] # filter for rows part of training set and their according columns
    y_train = data[data["train"] != ""]["train"] # filter for rows part of training set and their target column
    
    # extract test data
    x_test = data[data["test"] != ""][columns]
    y_test = data[data["test"] != ""]["test"]

    
    model,encoder = nn(x_train,y_train)
    # predict test dataset
    print("precicting test dataset...")
    pred = model.predict(x_test)
    pred = np.argmax(pred, axis=1)
    pred = encoder.inverse_transform(pred)
    acc = sklearn.metrics.accuracy_score(y_test,pred)
    print("Accuracy based on Columns",title," : ",round(acc,3))
    
    # predict full dataset
    print("precicting full dataset...")
    pred_full = model.predict(data[columns])
    pred_full = np.argmax(pred_full, axis=1)
    pred_full = encoder.inverse_transform(pred_full)
    data["pred"] = pred_full
    
    
    return(data)
    

In [121]:
def nn(x,y):
    
    y = np.array(y) 
    x = np.array(x)
    input_dim = x.shape[1]
        
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_y = encoder.transform(y)
    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_y)
    y = dummy_y
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

    
    # encode target variables
    #y_train = to_categorical(y_train)
    #y_test = to_categorical(y_test)
    count_classes = y_test.shape[1]
    #print("Count Classes:",count_classes)
    
    
    model = Sequential()
    model.add(Dense(400, activation='relu', input_dim=input_dim))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(7, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=13)
    return(model,encoder)

## Perform Predictions and Acccuracy assesment for train,test and complete dataset

In [ ]:
for combi in combinations[:]:
    print("\n\n\nPROCESSING COMBINATION:",combi)
    columns = combi[0]
    title = combi[1]
    complete_pred = workflow(columns,title,data=df)
    complete_pred.to_csv("NN_CASS"+str(title)+str(".csv"))




PROCESSING COMBINATION: (['r', 'g', 'b'], 'RGB')
Epoch 1/13
4860/4860 [==============================] - 17s 3ms/step - loss: 0.9014 - accuracy: 0.6677
Epoch 2/13
4860/4860 [==============================] - 17s 4ms/step - loss: 0.7403 - accuracy: 0.7174
Epoch 3/13
4860/4860 [==============================] - 16s 3ms/step - loss: 0.6960 - accuracy: 0.7321
Epoch 4/13
4860/4860 [==============================] - 15s 3ms/step - loss: 0.6788 - accuracy: 0.7371
Epoch 5/13
4860/4860 [==============================] - 17s 3ms/step - loss: 0.6664 - accuracy: 0.7417
Epoch 6/13
4860/4860 [==============================] - 18s 4ms/step - loss: 0.6587 - accuracy: 0.7458
Epoch 7/13
4860/4860 [==============================] - 17s 3ms/step - loss: 0.6506 - accuracy: 0.7482
Epoch 8/13
4860/4860 [==============================] - 16s 3ms/step - loss: 0.6452 - accuracy: 0.7506
Epoch 9/13
4860/4860 [==============================] - 17s 3ms/step - loss: 0.6438 - accuracy: 0.7504
Epoch 10/13
4860/4860